In [3]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Install TensorFlow and Transformers

In [1]:
!pip install -q tensorflow==2.1.0 --upgrade --ignore-installed
!pip install -q pytorch==1.5.0 --upgrade --ignore-installed
!pip install -q transformers==2.8.0

ERROR: tfx 0.21.4 has requirement absl-py<0.9,>=0.1.6, but you'll have absl-py 0.9.0 which is incompatible.
ERROR: tfx 0.21.4 has requirement apache-beam[gcp]<2.18,>=2.17, but you'll have apache-beam 2.20.0 which is incompatible.
ERROR: tfx 0.21.4 has requirement pyarrow<0.16,>=0.15, but you'll have pyarrow 0.17.1 which is incompatible.
ERROR: tfx 0.21.4 has requirement tensorflow-data-validation<0.22,>=0.21.4, but you'll have tensorflow-data-validation 0.22.0 which is incompatible.
ERROR: tfx 0.21.4 has requirement tensorflow-model-analysis<0.22,>=0.21.4, but you'll have tensorflow-model-analysis 0.22.1 which is incompatible.
ERROR: tfx 0.21.4 has requirement tensorflow-transform<0.22,>=0.21.2, but you'll have tensorflow-transform 0.22.0 which is incompatible.
ERROR: tfx 0.21.4 has requirement tfx-bsl<0.22,>=0.21.3, but you'll have tfx-bsl 0.22.0 which is incompatible.
ERROR: tfx-bsl 0.22.0 has requirement absl-py<0.9,>=0.7, but you'll have absl-py 0.9.0 which is incompatible.
ERROR: 

# Retrieve the Training Job Name

In [2]:
%store -r training_job_name

In [4]:
print('Previous training_job_name: {}'.format(training_job_name))

Previous training_job_name: tensorflow-training-2020-06-01-06-02-14-836


# Download the Trained Model

In [5]:
models_dir = './models'

In [6]:
# Download the model and output artifacts from AWS S3
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz $models_dir/model.tar.gz

download: s3://sagemaker-us-east-1-835319576252/tensorflow-training-2020-06-01-06-02-14-836/output/model.tar.gz to models/model.tar.gz


In [7]:
import tarfile
import pickle as pkl

#!ls -al ./models

tar = tarfile.open('{}/model.tar.gz'.format(models_dir))
tar.extractall(path=models_dir)
tar.close()

In [8]:
!ls -al $models_dir

total 1608704
drwxrwxr-x  4 ec2-user ec2-user       4096 Jun  1 22:43 .
drwxrwxr-x 13 ec2-user ec2-user       4096 Jun  1 22:43 ..
-rw-rw-r--  1 ec2-user ec2-user 1647289616 Jun  1 06:42 model.tar.gz
drwxr-xr-x  3 ec2-user ec2-user       4096 Jun  1 06:07 tensorflow
drwxr-xr-x  3 ec2-user ec2-user       4096 Jun  1 06:07 transformers


# Load the Model

In [9]:
transformer_model_dir = '{}/transformers/fine-tuned/'.format(models_dir)

!ls -al $transformer_model_dir

total 261692
drwxr-xr-x 2 ec2-user ec2-user      4096 Jun  1 06:38 .
drwxr-xr-x 3 ec2-user ec2-user      4096 Jun  1 06:07 ..
-rw-r--r-- 1 ec2-user ec2-user      1358 Jun  1 06:39 config.json
-rw-r--r-- 1 ec2-user ec2-user 267959068 Jun  1 06:39 tf_model.h5


In [10]:
cat $transformer_model_dir/config.json

{
  "_num_labels": 5,
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "dim": 768,
  "do_sample": false,
  "dropout": 0.1,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "min_length": 0,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "no_repeat_ngram_size": 0,
  "num_beams": 1,
  "num_return_sequences": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true

In [11]:
!pip install torch==1.5.0

     |████████████████████            | 471.1 MB 122.8 MB/s eta 0:00:03   |██▋                             | 61.8 MB 17.0 MB/s eta 0:00:41     |██████████████▎                 | 334.8 MB 105.6 MB/s eta 0:00:04     |█████████████████▌              | 410.1 MB 105.6 MB/s eta 0:00:04     |███████████████████             | 445.6 MB 122.8 MB/s eta 0:00:03

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |██████████████████████████▎     | 618.0 MB 128.4 MB/s eta 0:00:02

     |██████████████████████████▋     | 624.2 MB 128.4 MB/s eta 0:00:01

     |██████████████████████████▉     | 630.4 MB 128.4 MB/s eta 0:00:01

...

     |████████████████████████████▉   | 678.3 MB 128.4 MB/s eta 0:00:01

     |█████████████████████████████▏  | 684.2 MB 128.4 MB/s eta 0:00:01

     |█████████████████████████████▋  | 695.9 MB 135.0 MB/s eta 0:00:01

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 752.0 MB 2.1 kB/s 


In [12]:
!pip install transformers==2.8.0 --ignore-installed

  Using cached transformers-2.8.0-py3-none-any.whl (563 kB)
Processing /home/ec2-user/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45/sacremoses-0.0.43-cp36-none-any.whl
  Using cached tqdm-4.46.0-py2.py3-none-any.whl (63 kB)
  Using cached dataclasses-0.7-py3-none-any.whl (18 kB)
  Using cached numpy-1.18.4-cp36-cp36m-manylinux1_x86_64.whl (20.2 MB)
  Using cached filelock-3.0.12-py3-none-any.whl (7.6 kB)
     |████████████████████████████████| 128 kB 16.4 MB/s eta 0:00:01
  Using cached tokenizers-0.5.2-cp36-cp36m-manylinux1_x86_64.whl (3.7 MB)
  Using cached sentencepiece-0.1.91-cp36-cp36m-manylinux1_x86_64.whl (1.1 MB)
  Using cached requests-2.23.0-py2.py3-none-any.whl (58 kB)
  Using cached regex-2020.5.14-cp36-cp36m-manylinux2010_x86_64.whl (675 kB)
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Using cached joblib-0.15.1-py3-none-any.whl (298 kB)
     |███████████████████████████████

In [13]:
from transformers import DistilBertForSequenceClassification # PyTorch version

loaded_model = DistilBertForSequenceClassification.from_pretrained(transformer_model_dir,
                                                                     id2label={
                                                                       0: 1,
                                                                       1: 2,
                                                                       2: 3,
                                                                       3: 4,
                                                                       4: 5
                                                                     },
                                                                     label2id={
                                                                       1: 0,
                                                                       2: 1,
                                                                       3: 2,
                                                                       4: 3,
                                                                       5: 4
                                                                     },
                                                                  from_tf=True)

In [14]:
print(type(loaded_model))
print(loaded_model)

<class 'transformers.modeling_distilbert.DistilBertForSequenceClassification'>
DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
   

In [15]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

inference_device = -1 # CPU: -1, GPU: 0
print('inference_device {}'.format(inference_device))

inference_device -1


In [18]:
from transformers import TextClassificationPipeline

inference_pipeline = TextClassificationPipeline(model=loaded_model, 
                                                tokenizer=tokenizer,
                                                framework='tf',
                                                device=inference_device) 

In [19]:
review = """I loved it!  I will recommend this to everyone."""
print(review, inference_pipeline(review))

TypeError: forward() got an unexpected keyword argument 'training'

In [ ]:
review = """Really bad.  I hope they don't make this anymore."""
print(review, inference_pipeline(review))